<a href="https://colab.research.google.com/github/Neelima-sathi/Weather-Forecast-Agent/blob/main/WeatherForecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pydantic-ai==0.1.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.7/152.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.5/301.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.

In [24]:
import nest_asyncio
nest_asyncio.apply()

**Weather Forecast - Function Tool**

Custom Weather Forecast Tool - OWM
Get API keys from: https://home.openweathermap.org/api_keys

In [11]:
import requests

BASE_URL = "https://api.openweathermap.org/data/2.5/weather"
API_KEY = "748d9991a85c568aaa2c95aa1bd1ed33"

# custom weather forecast function
def find_weather(city: str) -> dict:
  """This function returns the current weatehr forecast for the given city"""
  units = "metric"
  params = {
      'q': city,
      'appid': API_KEY,
      'units': units
  }
  response = requests.get(BASE_URL, params=params)
  result = response.json()

  return result

In [8]:
op = find_weather("chennai")
print(op)

{'coord': {'lon': 80.2785, 'lat': 13.0878}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02d'}], 'base': 'stations', 'main': {'temp': 33.57, 'feels_like': 40.57, 'temp_min': 32.79, 'temp_max': 35.69, 'pressure': 1005, 'humidity': 64, 'sea_level': 1005, 'grnd_level': 1004}, 'visibility': 7000, 'wind': {'speed': 6.17, 'deg': 150}, 'clouds': {'all': 20}, 'dt': 1745919373, 'sys': {'type': 2, 'id': 2093220, 'country': 'IN', 'sunrise': 1745885932, 'sunset': 1745931204}, 'timezone': 19800, 'id': 1264527, 'name': 'Chennai', 'cod': 200}


**Pydantic Agent**

In [9]:
import os
import requests
from pydantic import BaseModel
from pydantic_ai import Agent, RunContext
from pydantic_ai.settings import ModelSettings

In [12]:
os.environ["GROQ_API_KEY"] = "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"

In [13]:
# Define the output schema
class WeatherForecast(BaseModel):
    location: str
    description: str
    temperature_celsius: float

In [21]:
weather_agent = Agent(
    model="groq:llama-3.1-8b-instant",
    model_settings=ModelSettings(temperature=0.2),
    output_type=str,
    system_prompt=("You are a helpful weather assistant. Use the 'get_weather_forecast' tool to find current weather "
        "conditions for any city. Provide clean and friendly answers.")
)

In [22]:
# weather forecast tool
@weather_agent.tool
def get_weather_forecast(ctx: RunContext, city: str) -> WeatherForecast:
    """
    Returns current weather forecast for a city using OpenWeatherMap API.
    """
    url = "https://api.openweathermap.org/data/2.5/weather"
    api_key = "748d9991a85c568aaa2c95aa1bd1ed33"
    params = {
        'q': city,
        'appid': api_key,
        'units': 'metric'
    }

    res = requests.get(url, params=params).json()

    return WeatherForecast(
        location=res["name"],
        description=res["weather"][0]["description"].capitalize(),
        temperature_celsius=res["main"]["temp"]
    )

In [23]:
question = input("🌤️ Ask about the weather: ")
result = weather_agent.run_sync(question)
print("\n📍 Forecast:", result.output)

🌤️ Ask about the weather: Chennai

📍 Forecast: It's a pleasant day in Chennai with a temperature of 33.43°C and a few clouds in the sky.
